In [92]:
import socket

In [93]:
server_ip = "192.168.0.100"  # Replace with your ESP32's IP
# server_ip = "192.168.0.201"  # Replace with your ESP32's IP
server_port = 12345

# Create a TC{} socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((server_ip, server_port))
client_socket.settimeout(20)

In [94]:


from PIL import Image, ImageSequence
import numpy as np

def load_gif_as_numpy_array(file_path):
    # Open the GIF file
    gif = Image.open(file_path)

    # List to store frames as NumPy arrays
    frames = []

    # Iterate through frames in the GIF
    for frame in ImageSequence.Iterator(gif):
        # Convert the frame to RGB or RGBA format if needed
        frame = frame.convert("RGB")
        frame = frame.resize((8, 8),resample=Image.NEAREST)
        # Append the frame as a NumPy array
        frames.append(np.array(frame))

    # Stack frames along a new dimension (frame index as first dimension)
    gif_array = np.stack(frames, axis=0,dtype=np.uint8)

    return gif_array


In [95]:
# Example usage
file_path = "../PixelArt/crazy.gif"  # Path to your GIF file
gif_array = load_gif_as_numpy_array(file_path)

# Print shape of the resulting NumPy array
print(f"GIF shape: {gif_array.shape}") 

datadict= {
    "frames": np.uint16(gif_array.shape[0]).tobytes(),
    "fps": np.uint16(30).tobytes(),
    "brightness":np.uint16(3).tobytes(),
    "color_correction": np.array([120,120,120],dtype=np.uint8).tobytes(),
    "data": np.reshape(gif_array,newshape=-1).tobytes(),
}


datadict["data"] = datadict["frames"]+datadict["fps"]+datadict["brightness"]+datadict["color_correction"]+datadict["data"]
datadict["data"]

GIF shape: (44, 8, 8, 3)


b',\x00\x1e\x00\x03\x00xxx!\x0b$\x0c\x04\r\x1d\x17\x1f!\x0b$\x0c\x04\r!\x0b$\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x1d\x17\x1f\x0c\x04\r!\x0b$o\x02}\xa1,\xb0\x85J\x8c\xc5\x13\xcf!\x0b$\x0c\x04\r\x1d\x17\x1f\x0c\x04\ro\x02}\xc5\x13\xcfo\x02}\x85J\x8c\x0c\x04\r\x0c\x04\r!\x0b$\x0c\x04\r\x85J\x8co\x02}\x85J\x8c\xa1,\xb0\x0c\x04\r\x0c\x04\r\x0c\x04\r!\x0b$\xc5\x13\xcf\xa1,\xb0\xc5\x13\xcfo\x02}\x0c\x04\r!\x0b$\x1d\x17\x1f\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x1d\x17\x1f\x0c\x04\r\x0c\x04\r!\x0b$\x1d\x17\x1f\x0c\x04\r\x0c\x04\r!\x0b$\x0c\x04\r\x0c\x04\r\xff\xff\xff\x0c\x04\r\x1d\x17\x1f!\x0b$\x0c\x04\r!\x0b$\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x0c\x04\r\x1d\x17\x1f\x0c\x04\r!\x0b$o\x02}\xa1,\xb0\x85J\x8c\xc5\x13\xcf!\x0b$\x0c\x04\r\x1d\x17\x1f\x0c\x04\ro\x02}\xc5\x13\xcfo\x02}\x85J\x8c\x0c\x04\r\x0c\x04\r!\x0b$\x0c\x04\r\x85J\x8co\x02}\x85J\x8c\xa1,\xb0\x0c\x04\r\x0c\x04\r\x0c\x04\r!

In [96]:
client_socket.sendall(datadict["data"])
# Receive the response from the server
data, addr = client_socket.recvfrom(1024)
print(f"Response from server: {data.decode()}")
print("Artwork Published!")

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host